# 函数式编程
前情提要：
- 迭代：指能够通过for来遍历，如list，tuple
    - 用于判断是否是可迭代对象：isinstance([], Iterable)
- 列表生成式：[x*x for x in range(1,11) if x % 2 == 0], [m + n for m in 'ABC' for n in 'XYZ']
- 生成器：如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator
    - L = [x * x for x in range(10)]#这是一个列表生成式
    - g = (x * x for x in range(10))#这是一个生成器
        - next(g)#或者直接用for遍历，因为生成器也是可迭代对象
        - 很适用于像斐波那契那样，不能通过列表生成式写出来，但是可以一个一个打出来的东西
        - 当在函数中把print(b)写成yield(b),则函数便成了生成器函数，生成器的用法是o=add() next(o)#因为每次调用生成器函数，都会生成一个生成器对象
- 迭代器：
    - 生成器都是Iterator对象，但list、dict、str虽然是Iterable，却不是迭代器
    - 把list、dict、str等Iterable变成Iterator可以使用iter()函数


## 高阶函数

### map/reduce
- map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回

In [2]:
def f(x):
    return x*x
r = map(f,[1,2,3,4])#结果r是一个迭代器
list(r)

[1, 4, 9, 16]

- reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算

In [4]:
from functools import reduce
def add(x, y):
     return x + y

reduce(add, [1, 3, 5, 7, 9])

25

In [1]:
s = "123.145"
s= s.split('.')
s

['123', '145']

### filter()函数
Python内建的filter()函数用于过滤序列。

和map()类似，filter()也接收一个函数和一个序列。和map()不同的是，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

filter返回的是一个iterator，要用list获得结果

In [2]:
def not_empty(s):
    return s and s.strip()#s.strip删除字符串头或尾的空格（括号里可以指定删除的字符）

list(filter(not_empty, ['A', '', 'B', None, 'C', '  ']))

['A', 'B', 'C']

### sorted()
Python内置的sorted()函数就可以对list进行排序：  
此外，sorted()函数也是一个高阶函数，它还可以接收一个key函数来实现自定义的排序，例如按绝对值大小排序：

In [5]:
sorted([36, 5, -12, 9, -21], key=abs)

[5, 9, -12, -21, 36]

In [6]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower)

['about', 'bob', 'Credit', 'Zoo']

In [8]:
#要进行反向排序，不必改动key函数，可以传入第三个参数reverse=True：
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower, reverse=True)

['Zoo', 'Credit', 'bob', 'about']

## 返回函数
- 闭包：当一个函数返回了一个函数后，其内部的局部变量还被新函数引用；  
    - 另一个需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了f()才执行。  
- 返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。  
- 使用闭包时，对外层变量赋值前，需要先使用nonlocal声明该变量不是当前函数的局部变量。

## 匿名函数 

In [12]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

## 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数。 
函数对象有一个__name__属性，可以拿到函数的名字：  
现在，假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。

本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：  
``` python
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper
```
    观察上面的log，因为它是一个decorator，所以接受一个函数作为参数，并返回一个函数。我们要借助Python的@语法，把decorator置于函数的定义处：  
``` python
@log
def now():
    print('2015-3-25')
```
    调用now()函数，不仅会运行now()函数本身，还会在运行now()函数前打印一行日志：  
```python
    >>> now()
call now():
2015-3-25
```
    把@log放到now()函数的定义处，相当于执行了语句：  

    `now = log(now)`

## 偏函数
int()函数还提供额外的base参数，默认值为10。如果传入base参数，就可以做N进制的转换：
```python
    >>> int('12345', base=8)
    5349
    >>> int('12345', 16)
    74565
```


In [17]:
#functools.partial就是帮助我们创建一个偏函数的，不需要我们自己定义int2()，可以直接使用下面的代码创建一个新的函数int2：
import functools
int2 = functools.partial(int, base=2)
int2('1000000')

64

- 所以，简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

- 注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值：

# module模块
为了编写可维护的代码，我们把很多函数分组，分别放到不同的文件里，这样，每个文件包含的代码就相对较少，很多编程语言都采用这种组织代码的方式。在Python中，一个.py文件就称之为一个模块（Module）。  
你也许还想到，如果不同的人编写的模块名相同怎么办？为了避免模块名冲突，Python又引入了按目录来组织模块的方法，称为包（Package）。 
``` 
    mycompany
    ├─ __init__.py
    ├─ abc.py
    └─ xyz.py 
```
请注意，每一个包目录下面都会有一个__init__.py的文件，这个文件是必须存在的，否则，Python就把这个目录当成普通目录，而不是一个包。__init__.py可以是空文件，也可以有Python代码，因为__init__.py本身就是一个模块，而它的模块名就是mycompany

类似__xxx__这样的变量是特殊变量，可以被直接引用，但是有特殊用途，比如上面的__author__，__name__就是特殊变量，hello模块定义的文档注释也可以用特殊变量__doc__访问，我们自己的变量一般不要用这种变量名；  

类似_xxx和__xxx这样的函数或变量就是非公开的（private），不应该被直接引用，比如_abc，__abc等；  

之所以我们说，private函数和变量“不应该”被直接引用，而不是“不能”被直接引用，是因为Python并没有一种方法可以完全限制访问private函数或变量，但是，从编程习惯上不应该引用private函数或变量。  

# OOP:面向对象编程
## 类和实例
- 类：class Student(object):定义了一个继承自object的类student；
- 实例：bart = Student()；
- 通过定义一个特殊的__init__方法，在创建实例的时候，就把name，score等属性绑上去：
```
class Student(object):

    def __init__(self, name, score):#self表示创建的实例本身
        self.name = name
        self.score = score
```

## 访问限制
如果要让内部属性不被外部访问，可以把属性的名称前加上两个下划线__，在Python中，实例的变量名如果以__开头，就变成了一个私有变量（private），只有内部可以访问，外部不能访问。
```
class Student(object):

    def __init__(self, name, score):
        self.__name = name
        self.__score = score

    def print_score(self):
        print('%s: %s' % (self.__name, self.__score))
```
- 改完后，对于外部代码来说，没什么变动，但是已经无法从外部访问实例变量.__name和实例变量.__score了
- 如果又要允许外部代码修改score怎么办？可以再给Student类增加set_score方法：
```
class Student(object):
    ...

    def set_score(self, score):
        if 0 <= score <= 100:
            self.__score = score
        else:
            raise ValueError('bad score')
```
- 需要注意的是，在Python中，变量名类似__xxx__的，也就是以双下划线开头，并且以双下划线结尾的，是特殊变量，特殊变量是可以直接访问的，不是private变量，所以，不能用__name__、__score__这样的变量名
- _name，这样的实例变量外部是可以访问的，但是，按照约定俗成的规定，当你看到这样的变量时，意思就是，“虽然我可以被访问，但是，请把我视为私有变量，不要随意访问”。双下划线开头的实例变量是不是一定不能从外部访问呢？其实也不是。不能直接访问__name是因为Python解释器对外把__name变量改成了_Student__name，所以，仍然可以通过_Student__name来访问__name变量.


## 获取对象信息
- type（）：type(123)==int
    - 但如果要判断一个对象是否是函数怎么办？可以使用types模块中定义的常量：
    ```
    >>> import types
    >>> def fn():
    ...     pass
    ...
    >>> type(fn)==types.FunctionType
    ```
- isinstance():判断是否是某个类的实例，或者是继承关系也可
- dir()：如果要获得一个对象的所有属性和方法，可以使用dir()函数，它返回一个包含字符串的list
    - 类似__xxx__的属性和方法在Python中都是有特殊用途的，比如__len__方法返回长度。在Python中，如果你调用len()函数试图获取一个对象的长度，实际上，在len()函数内部，它自动去调用该对象的__len__()方法，所以，下面的代码是等价的：
    ```
    >>> len('ABC')
    3
    >>> 'ABC'.__len__()
    3
    ```
    - hasattr(obj, 'x') # 有属性'x'吗？
    - setattr(obj, 'y', 19) # 设置一个属性'y'
    - getattr(obj, 'y') # 获取属性'y'
    - getattr(obj, 'z', 404) # 获取属性'z'，如果不存在，返回默认值404

## 实例属性和类属性
- 在类中定义个属性叫做类属性，而通过self或.创建的属性是实例属性，
- 两者不可同名，不然实例属性会覆盖类属性

## 使用_ _slots_ _
- 限制实例的属性，只允许对Student实例添加name和age属性。  
- 为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的__slots__变量，来限制该class实例能添加的属性：
    ```
    class Student(object):
        __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称
    ```
- 使用__slots__要注意，__slots__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的：
- 除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__。

## 使用@property
在设置学生成绩的类中，需要设置一个set_score方法，以正确地修改score，但是太过麻烦，可以使用@property
- 装饰器（decorator）可以给函数动态加上功能
- Python内置的@property装饰器就是负责把一个方法变成属性调用的，使得再外面对实例进行设置时可以按照方法设置该属性
    ```
    class Student(object):

    @property
    def score(self):#等价于get_score
        return self._score
    @score.setter
    def score(self, value):# 等价于set_score
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

    ```